In [3]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process.kernels import Matern
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [45]:
def data(node_num):
  df = pd.read_csv('Node_50{}.csv'.format(node_num))

  # Split data into train and test sets
  train = df.dropna()

  hum_out = df[df['humidity'].isnull()]
  temp_out = df[df['temperature'].isnull()]
  X1=train[['ID','humidity']]
  Y1=train['temperature']

  X1_train,X1_test,y1_train,y1_test=train_test_split(X1,Y1,test_size=0.2)


  X2=train[['ID','temperature']]
  Y2=train['humidity']
  X2_train,X2_test,y2_train,y2_test=train_test_split(X2,Y2,test_size=0.2)
  # Define kernel function for GPR
  kernel = Matern(nu=1.5)
  # Train GPR on the non-missing values
  gpr_temp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1)
  gpr_temp.fit(X1_train,y1_train)
  y1_pred=gpr_temp.predict(X1_test)
  mse1=mean_squared_error(y1_pred,y1_test)
  print("mean square error for node {} (input temperature) is: {} ".format(node_num,mse1))
  print('\n')
  gpr_humid = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1)
  gpr_humid.fit(X2_train,y2_train)
  y2_pred=gpr_humid.predict(X2_test)
  mse2=mean_squared_error(y2_pred,y2_test)
  print("mean square error for node {} (input humidity) is: {} ".format(node_num,mse2))
  print('\n\n\n')
  # Predict the missing values using GPR
  temperature= gpr_temp.predict(temp_out[[ 'ID','humidity']])
  humidity = gpr_humid.predict(hum_out[[ 'ID','temperature']])
  ID_temp=temp_out['ID']
  ID_humid=hum_out['ID']
  return ID_temp,temperature,ID_humid,humidity



In [33]:
ID_temp_1,temp_1,ID_humid_1,humid_1=data(1)


Index(['ID', 'temperature'], dtype='object')
0    36.47
1    35.52
Name: humidity, dtype: float64
Index(['ID', 'temperature'], dtype='object')
1309    70.36
1125    34.42
Name: humidity, dtype: float64
mean square error for node 1 (input temperature) is: 0.37674823746082514 


mean square error for node 1 (input humidity) is: 5.890096577308475 






In [34]:
ID_temp_2,temp_2,ID_humid_2,humid_2=data(2)

Index(['ID', 'temperature'], dtype='object')
0    30.63
1    31.11
Name: humidity, dtype: float64
Index(['ID', 'temperature'], dtype='object')
839    63.117
502    55.390
Name: humidity, dtype: float64
mean square error for node 2 (input temperature) is: 0.4384583355047268 


mean square error for node 2 (input humidity) is: 2.4750638230908653 






In [36]:
ID_temp_5,temp_5,ID_humid_5,humid_5=data(5)

mean square error for node 5 (input temperature) is: 0.18809245064773425 


mean square error for node 5 (input humidity) is: 3.5501370562933348 






In [37]:
ID_temp_7,temp_7,ID_humid_7,humid_7=data(7)

mean square error for node 7 (input temperature) is: 0.2889311912416171 


mean square error for node 7 (input humidity) is: 4.062736699676026 






In [38]:
ID_temp_8,temp_8,ID_humid_8,humid_8=data(8)

mean square error for node 8 (input temperature) is: 0.2625555078531739 


mean square error for node 8 (input humidity) is: 2.2612033757226775 






In [42]:
ID=np.concatenate((ID_temp_1,ID_humid_1,ID_temp_2,ID_humid_2,ID_temp_5,ID_humid_5,ID_temp_7,ID_humid_7,ID_temp_8,ID_humid_8),axis=0)
predictions=np.concatenate((temp_1,humid_1,temp_2,humid_2,temp_5,humid_5,temp_7,humid_7,temp_8,humid_8),axis=0)

In [43]:
sub=pd.DataFrame()
sub['ID']=ID
sub['predictions']=predictions
sub.to_csv('sub.csv')

In [44]:
sub.head()

,ID,predictions
0,20,23.294183
1,119,22.801409
2,149,24.255012
3,155,24.333413
4,163,24.090154
